<a id='data-structure'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Data Model </span> </p>

This notebook has the main structures used to hold data - This is usually called the **data model**.

# References
This section initializes libraries and code from other notebooks, which will be used here, as well as information needed for querying data from the database.

## Libraries

In [0]:
#r "nuget:Systemorph.Activities,1.6.5"
#r "nuget:Systemorph.Arithmetics,1.6.5"
#r "nuget:Systemorph.Workspace,1.6.4"
#r "nuget:Systemorph.InteractiveObjects,1.6.5"
//#r "nuget:Systemorph.SharePoint,1.6.5"
//#r "nuget:Systemorph.OneDrive,1.6.5"
#r "nuget:Systemorph.Scopes,1.6.5"
#r "nuget:Systemorph.Import,1.6.7"
#r "nuget:Systemorph.Test,1.6.5"
#r "nuget:Systemorph.Export,1.6.7"
#r "nuget:Systemorph.DataSetReader,1.6.6"
#r "nuget:Systemorph.DataSource,1.6.4"
#r "nuget:Systemorph.DataSource.Conversions,1.6.4"
#r "nuget:Systemorph.Reporting,1.6.5"
#r "nuget:Systemorph.Charting,1.6.5"
#r "nuget:Systemorph.SchemaMigrations,1.6.4"

## Usings

In [0]:
using System.ComponentModel.DataAnnotations;
using System.ComponentModel.DataAnnotations.Schema;
using Systemorph.Vertex.Grid.Model;
using Systemorph.Vertex.Workspace;
using Systemorph.Vertex.Activities;
using Systemorph.Vertex.Import;
using static Systemorph.Vertex.Arithmetics.ArithmeticOperations;

## Notebooks

In [0]:
#!import "../Constants/Enums"
#!import "../Constants/Consts"
#!import "../Constants/Validations"

# Data Infrastructure

<a id='base-interfaces'></a>
## Base Interfaces
The following interfaces are used to define the granularity at which data is stored.

For instance, the IKeyed interface corresponds to the lowest granularity which is simply a unique identifier for a certain generic piece of data.

In [0]:
public interface IKeyed
{   
    public Guid Id { get; init; }
}

The interface IPartition identifies the partitions in the system.

In [0]:
public interface IPartition : IKeyed {}

The interface IPartitioned simply assigns a unique identifier for elements of a data partition - The exact definition of such elements can be found in the [partition](#partition) section below.

In [0]:
public interface IPartitioned
{
    public Guid Partition { get; init; }
}

The interface IWithYearAndMonth allows to specify to which year and month a certain piece of data applies to.

In [0]:
public interface IWithYearAndMonth
{
    public int Year { get; init; }
    
    public int Month { get; init; }
}

The interface IWithYearMonthAndScenario allows to identify the year, month and scenario associated to the data.

In [0]:
public interface IWithYearMonthAndScenario : IWithYearAndMonth
{
    public string Scenario { get; init; }
}

<a id='abstract-classes'></a>
## Abstract Classes
These classes are not reflected in the database, i.e. are not persisted.

In [0]:
public abstract record KeyedRecord : IKeyed {
    [Key]
    [NotVisible]     
    public Guid Id { get; init; }
}

In [0]:
public abstract record KeyedDimension : INamed {
    [Key]
    [IdentityProperty]
    [StringLength(50)]
    public string SystemName { get; init; }
    
    [NotVisible]
    public string DisplayName { get; init; }
}

In [0]:
public abstract record KeyedOrderedDimension : KeyedDimension, IOrdered {
    [NotVisible]
    public int Order { get; init; }
}

In [0]:
public abstract record KeyedOrderedDimensionWithExternalId : KeyedOrderedDimension {
    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] ExternalId { get; init; }
}

<a id='dimensions'></a>
# Dimensions

Dimensions are data which typically can be given a short name, here called 'System Name'. They also change slowly with time. A typical example is currencies, and examples of System Names are EUR, CHF, etc.

The advantages of using Dimensions are:
- The data is easier to maintain (the data is said to be 'normalized');
- The business logic is easier to read;
- Easier scructuring of the data, e.g. in a data-cube.

<a id='hierarchical-dimension-with-level'></a>
## Hierarchical Dimension With Level
Hierarchical dimensions are useful to define for example the legal structure of a company (reporting nodes and their parents).

Below we introduce the concept of a hierarchical dimension with level, which is a simple way to keep track of the "level" in a hierarchical dimension.

In [0]:
public record HierarchicalDimensionWithLevel(string SystemName, string DisplayName, string Parent, int Level) : IHierarchicalDimension;

<a id='amount-type'></a>
## Amount Type

Amount Type is a dimension used to describe the nature of numbers. For example: Numbers associated to the payment of premiums have Amount Type 'Premiums'; Numbers associated to the payment of claims have Amount Type 'Claims'; Numbers associated to expenses have Amount Type 'Expenses'. 

It is defined as a hierarchical dimension so that higher-granularity types can be introduced. For example: 'Claim Expenses' have parent 'Claims'.

Amount Types are a central piece in the IFRS 17 application and are used for instance to name the different types of cash flows. For example: There might be cash flows for 'Premiums'; Cash flows for 'Claims', etc. In case the mapping between cash flow and amount type is non-trivial, then specific mappings are introduced through the use of <code>ExternalId</code>.

In [0]:
public record AmountType : KeyedOrderedDimensionWithExternalId, IHierarchicalDimension
{
    [Dimension(typeof(AmountType))]
    public string Parent { get; init; }
    
    [Dimension(typeof(PeriodType))]
    public PeriodType PeriodType { get; init; }
}

We distinguish between amount types used for cash flows and actuals (**Present Value Amount Types**), and amount types used for deferrable values (**Deferrable Amount Types**). In particular, deferrable amount types are for costs and expenses that can be deferred to the future. These amounts are treated separately from the Present Value Amount Types because they are not considered for the calculation of Contractual Service Margin and Loss Component.

In [0]:
public record DeferrableAmountType : AmountType {}

<a id='risk-driver'></a>
## Risk Driver

Risk drivers are used to distinguish between the different Risk Adjustment numbers. For instance: Mortality, Disability, Longevity, etc.

When only total risk adjustment values are used, then there is no need to use risk drivers.

In [0]:
public record RiskDriver : KeyedOrderedDimension, IHierarchicalDimension
{
    [Dimension(typeof(RiskDriver))]
    public string Parent { get; init; }
}

<a id='estimate-type'></a>
## Estimate Type

Estimate Type is a [dimension](#dimensions) used to distinguish the context to which numbers belong to. For instance: Numbers for present values of Best Estimate have Estimate Type 'Best Estimate'; Numbers for present values of Risk Adjustments have Estimate Type 'Risk Adjustment'.

It holds information about the <code>StructureType</code> which is used in the [ReportScopes](Report/ReportScopes#data-transformation) to efficiently transform the data before visualization. To this scope it is relevant to distinguish between <code>None</code> and defined <code>StructureType</code>.

In [0]:
public record EstimateType : KeyedOrderedDimensionWithExternalId
{
    public InputSource InputSource { get; init; }
    
    public StructureType StructureType { get; init; }

    [Dimension(typeof(PeriodType))]
    public PeriodType PeriodType { get; init; }
}

<a id='novelty'></a>
## Novelty

Novelty is a [dimension](#dimensions) used to identity the novelty of the business. Typical examples of Novelty are New Busines, In-Force, and Combined.

In [0]:
public record Novelty : KeyedOrderedDimension {}

<a id='variable-type'></a>
## Variable Type

In [0]:
public record VariableType : KeyedOrderedDimension, IHierarchicalDimension
{    
    public string Parent { get; init; }
}

<a id='aoc-type'></a>
### AoC Variable Type

The AoC Type describes the type of the Analysis of Change step - For example: Amortization, Cash flow, Combined Liabilities etc.

The **Combined Liability** AoC Type aims in capturing the difference between the sum of all relevant previous AoC Steps and the end of period AoC Step. Unexplained changes (not captured by any other defined AoC Step), are then visible in this AoC Step. Note that the nominal cash flows input in the cash flow input file corresponds to cash flow projection for the end of period step.
In addition, this step distinguishes itself from the others for:
- It is a mandatory AoC Type as it is used to trigger the Run off of the cash flow. This is achieved by providing a cash flow of 0s. 
- It is the first AoC Type for the Combined Novelty and therefore it carries the contribution of combining In force and New business.
- Its present value is computed as telescoping difference with the last AoC Step for the In-Force novelty and the New Business novelty.
- In the technical margin calculation it is used to merge the profitability of the In-Force and New business components, which may result in a forced switch.
- If ReleasePattern is provided in the Cash Flows it must be provided for this AocType.

In [0]:
public record AocType : VariableType
{    
    [Dimension(typeof(AocType))]
    public string Parent { get; init; }
}

In [0]:
public record AocStep(string AocType, string Novelty){}

<a id='pnl-variable-type'></a>
### PnL Variable Type

The PnlVariableType is used to describe the various profit and loss sections and the corresponding hierarchy.

In [0]:
public record PnlVariableType : VariableType {}

<a id='bs-variable-type'></a>
## Balance Sheet Variable Type

The BsVariableType is used to descibe the Analysis of change of the Balance Sheet reports.

In [0]:
public record BsVariableType : VariableType {}

## Accounting Variable Type
The AccountingVariableType is used to describe the accounting events that are posted in the accounting output file.

In [0]:
public record AccountingVariableType : VariableType {}

<a id='scenario'></a>
## Scenario
The <code>Scenario</code> record holds the various Scenarios used for Sensitivity analysis. 

Any Scenario-case can be defined with this record by providing a SystemName and a DisplayName. During data collection phase the Scenario column of the [main](../Import/Importers#parse-the-main-tab) table of the input can be populated with the SystemName of the desired Scenario. The default Scenario (i.e. the default one, with no-stress situations applied) is referred to as 'Best Estimate' and its identifier is a null string, allowing the input files to not specify any value under the column Scenario or to lack the column itself.

In [0]:
public record Scenario : KeyedDimension {}

### Scenario and inputs

Under a particular Scenario, several data types can be imported (e.g. nominal cash flows, parameters, yield-curvers, etc.). However, we suggest arranging a different Scenario for every desired type of stress to facilitate the analysis of the results.

### Dependecy with Best Estimate scenario: same period

When a file is imported for a specific Scenario, the calculation engine integrates the set of inputs taking the remaining from the best estimate scenario. In this way the user is not required to input again all data for each Scenario calculation but only the file with the stressed input. This is achieved through our system of [Relaxed Queries](../Utils/Queries). The assumption here is that the Best Estimate scenario is the first to be imported, and the stressed scenarios follow. 

In the case of re-import, the Engine considers the dependency between each scenario and the Best Estimate scenario. In this case, one import automatically triggers calculation in several partitions allowing all dependant cases to be updated. For more details on how the calculation is performed refer to [Calculate IFRS Variables: for all scenarios](../Import/Importers#calculate-ifrs-variables-for-all-scenarios).

The only exception to this is applied to time dependency. When the user imports figures for periods $P$ and $P+1$ are imported for all scenarios and a new input for Best Estimate period $P$ is provided, only the scenarios for period $P$ are automatically updated. A manual update of period $P+1$ is then required to update the figures of $P+1$. We consider the case of restating previous periods particularly sensitive and defer to the user the resposability to ensure that all results are up to date. 

### Dependecy with Best Estimate scenario: across periods

When a stress scenario is imported for a period $P$ and a previous period $P-1$ is available, the End of Period values ($P-1$) of the Best Estimate scenario is considered as Beginning of Period ($P$) for the stressed and no-stress scenarios. This implies that:
1. scenarios can be occasionally not be calculated without impacting their calculation in future periods,
2. new scenarios can be added at any time in a production environment,
3. in each period the scenario depends only on the perturbation provided in the period and is not applied on top of the previous period perturbation.

<a id='line-of-business'></a>
## Line Of Business

Line Of Business is a dimension used to identity insurance business according to usual business classifications (usually a market or company-specific business classification representing a convolution of risk covered, 1st party / 3rd party cover type, product type, etc...)
Typical examples of Line of Business are Motor, Property, Endowment, etc... This is a hierarchical dimension.

In [0]:
public record LineOfBusiness : KeyedOrderedDimension, IHierarchicalDimension
{
    [Dimension(typeof(LineOfBusiness))]
    public string Parent { get; init; }
}

<a id='currency'></a>
## Currency

Currency is a dimension used to identity the currency.
Typical examples of Currency are their ISO codes: EUR, CHF, USD, etc.

In [0]:
public record Currency : KeyedDimension {}

<a id='economic-basis'></a>
## Economic Basis

The Economic Basis defines the discount type which is used in calculations. Examples are: Nominal, Current and Locked.

In [0]:
public record EconomicBasis : KeyedDimension {}

<a id='valuation-approach'></a>
## Valuation Approach
This dimension holds the various methodologies used in the calculations: BBA, VFA and PAA.

In [0]:
public record ValuationApproach : KeyedDimension {}

<a id='liability-type'></a>
## Liability Type
The Liability Type distinguishes betweeen e.g. liabilities for incurred claims and liabilities for remaining coverage.

In [0]:
public record LiabilityType : KeyedOrderedDimension, IHierarchicalDimension
{
    [Dimension(typeof(LiabilityType))]
    public string Parent { get; init; }
}

<a id='oci-type'></a>
## OCI Type
Other Comprehensive Income (OCI) Type describes how the financial contributions to the IFRS 17 Financial Performance is apportioned between the P&L and the OCI sections.

In [0]:
public record OciType : KeyedDimension {}

<a id='profitability'></a>
## Profitability

Profitability is a dimension used in IFRS 17 to describe the profitability of a Group of Contracts (GIC/GRIC).
Typical examples of Profitability are Profitable, Non-Profitable, etc.

In [0]:
public record Profitability : KeyedDimension {}

<a id='partner'></a>
## Partner
Partner is the counterparty of a reinsurance contract.

In [0]:
public record Partner : KeyedDimension {}

<a id='credit-risk-rating'></a>
## Credit Risk Rating
This holds the credit risk rating of each Partner.

In [0]:
public record CreditRiskRating : KeyedDimension {}

<a id='reporting-node'></a>
## Reporting Node
A Reporting Node is an element in the reporting structure of the company. It has a hierarchical structure and the end nodes is the level at which data is imported.

In [0]:
public record ReportingNode : KeyedDimension, IHierarchicalDimension 
{
    [Dimension(typeof(ReportingNode))]
    public string Parent { get; init; }
    
    [Required]
    [Dimension(typeof(Currency))]
    public virtual string Currency { get; init; }
}

<a id='projection-configuration'></a>
## Projection Configuration

Projections are configured by defining the following 2 parameters:

- <b>Shift</b> defines how many months are in the cash flow file before the first value of the period in interest.<br> 
- <b>TimeStep</b> defines how many months are in the period of interest. <br>

Both <b>Shift</b> and <b>TimeStep</b> are measured in terms of number of months.
For instance, for the current year and year-to-date view we have $\rm{Shift}=0$ and $TimeStep=3$ for the first quarter, $TimeStep=6$ for the 2nd quarter and so on.

In [0]:
public record ProjectionConfiguration : KeyedOrderedDimension
{
    [IdentityProperty]
    public int Shift { get; init; }
    [IdentityProperty]
    public int TimeStep { get; init; }
}

<a id='parameters'></a>
# Parameters

<a id='aoc-configuration'></a>
## AoC Step Configuration

The record below holds parameters over time for each combination of AoC Type and Novelty. In particular:
- <code>DataType</code> identifies between Calculated AoC Steps and Optional inputs or Mandatory inputs.
- <code>InputSource</code> identifies whether an AoC Step is relevant for Actuals, Cash flow or both calculations.
- <code>StructureType</code> identifies the analysis of change chain to which the AoC Step is used.
- <code>FxPeriod</code> identifies which fx rate should be used.
- <code>YcPeriod</code> identifies which Yield Curve should be used (from current or previous period).
- <code>CdrPeriod</code> identifies which Credit Default Rate should be used (from current or previous period).
- <code>ValuationPeriod</code> identifies which value from the Cash flow corresponds to the Present Value.
- <code>RcPeriod</code> identifies which Reinsurance Coverage should be used (from current or previous period).

<br>The values for each parameter can be found in [Enums Notebook](../Constants/Enums).

In [0]:
public record AocConfiguration : KeyedRecord, IWithYearAndMonth, IOrdered
{
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Month { get; init; }
    
    [IdentityProperty]
    [Dimension(typeof(AocType))]
    public string AocType { get; init; }
    
    [IdentityProperty]
    [Dimension(typeof(Novelty))]
    public string Novelty { get; init; }
    
    [Dimension(typeof(DataType))]
    public DataType DataType { get; init; }
    
    [Dimension(typeof(StructureType))]
    public StructureType StructureType { get; init; }

    [Dimension(typeof(InputSource))]
    public InputSource InputSource { get; init; }
    
    [Dimension(typeof(FxPeriod))]
    public FxPeriod FxPeriod { get; init; }
    
    [Dimension(typeof(PeriodType), nameof(YcPeriod))]
    public PeriodType YcPeriod { get; init; }
    
    [Dimension(typeof(PeriodType), nameof(CdrPeriod))]
    public PeriodType CdrPeriod { get; init; }
    
    [Dimension(typeof(ValuationPeriod))]
    public ValuationPeriod ValuationPeriod { get; init; }
    
    [Dimension(typeof(PeriodType), nameof(RcPeriod))]
    public PeriodType RcPeriod { get; init; }
    
    [NotVisible]
    public int Order { get; init; }
}

<a id='exchange-rate'></a>
## Exchange Rate
The record below holds the Exchange Rates over time and for each exchange type ([FxType](./Constants/Enums)). Exchange Rate must be updated every reporting period as part of the close preparation. In general, the last month of the reporting period should be used to import the new data for the period. 

In case the Exchange Rate for the current period is missing, an error is returned. 

In [0]:
public record ExchangeRate : KeyedRecord, IWithYearMonthAndScenario
{    
    [Required]
    [IdentityProperty]
    [Dimension(typeof(Currency))]
    public string Currency { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Month { get; init; }

    [IdentityProperty]
    [Required]
    public FxType FxType { get; init; }

    public double FxToGroupCurrency { get; init; }

    [IdentityProperty]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

<a id='credit-default-rate'></a>
## Credit Default Rate
The record below holds the Credit Default Rates over time for each [Credit Risk Rating](#credit-risk-rating). Credit Default Rate can be updated every month as part of the close preparation. In general, the last month of the reporting period should be use to import the new data for the period. 

The Credit Default Rate values must be entered with monthly granularity, i.e. every value corresponds to the default rate for a certain month. The first month entered in column Values0 always corresponds to January of the year entered in column <code>Year</code>. The last value is considered valid also for the following years (for which no value has been provided). 

The latest Credit Default Rate available in the system with Year and Month earlier or equal to Year and Month of the closing period will be used as the current value during calculation. When the current period Credit Default Rate differs from previous period value, the analysis of change for the Credit Default Rate amount type computes the contribution of such update as an additional step (Credit Risk Update). During calculation we approximate the Credit Default Rate to be constant over time as considering all projected values only accounts for marginal changes. 

In [0]:
public record CreditDefaultRate : KeyedRecord, IWithYearMonthAndScenario
{    
    [Required]
    [IdentityProperty]
    [Dimension(typeof(CreditRiskRating))]
    public string CreditRiskRating { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Month { get; init; }
    
    [Conversion(typeof(PrimitiveArrayConverter))]
    public double[] Values { get; init; }

    [IdentityProperty]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

<a id='yield-curve'></a>
## Yield Curve
The record below holds the forward rate Yield Curves across time for each [Currency](#currency) and [Scenario](#scenario).
Yield curves can be updated every month as part of the close preparation. In general, the last month of the reporting period should be used to import the new data for the period. 

The Yield Curve values must be entered with **yearly** granularity, i.e. every value corresponds to the rate of the whole year starting from the year entered in column <code>Year</code>. The last value is considered valid also for the following years (for which no value has been provided). 

During calculation, the Yield Curve values are retrieved from the DataSource. The Yield Curve Name defined for each group-of-contracts will be used to match the Name of the Yield Curve to be considered. This occurs only for the curve that is used in the computation of the Contractual Service Margin (locked-in for BBA valuation approach, current otherwise). The calculation engine allows the update of the locked-in curve during the entire inception year of the group-of-contracts. When the locked-in Yield Curve is used, the latest Yield Curve available in the system with Year earlier or equal to the inception Year of the group-of-contracts will be used. When the current Yield Curve is used in the calculation, the latest Yield Curve available in the system with Year and Month earlier or equal to Year and Month of the closing period will be used. When the current period Yield Curve differs from previous period value, the analysis of change computes the contribution of such update as an additional step (Yield Curve Update).

In [0]:
public record YieldCurve : KeyedRecord, IWithYearMonthAndScenario
{    
    [Required]
    [IdentityProperty]
    [Dimension(typeof(Currency))]
    public string Currency { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Month { get; init; }

    [IdentityProperty]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }

    [IdentityProperty]
    public string Name { get; init; }
    
    [Conversion(typeof(PrimitiveArrayConverter))]
    public double[] Values { get; init; }
}

<a id='partner-rating'></a>
## Partner Rating
The record below holds the [Credit Risk Rating](#credit-risk-partner) for each [Partner](#partner) across time. Partner Rating can be updated every month as part of the close preparation. In general, the last month of the reporting period should be use to import the new data for the period. The Credit Default Rates for each rating is defined through the import of [Credit Default Rates](#credit-default-rate).

In [0]:
public record PartnerRating : KeyedRecord, IWithYearMonthAndScenario
{    
    [Required]
    [IdentityProperty]
    [Dimension(typeof(Partner))]
    public string Partner { get; init; }

    [Required]
    [Dimension(typeof(CreditRiskRating))]
    public string CreditRiskRating { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Month { get; init; }

    [IdentityProperty]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

<a id='partitions'></a>
# Partitions
The notion of Partition is used internally to the calculation engine to define small but meaningful sets of data, called partitions. Partitions are then used to optimize the extraction of data from the database, i.e. only the data for a certain partition is retrieved. For example, a specific combination of Year and Month defines a partition.

In [0]:
public abstract record IfrsPartition : IPartition {
    [Key]
    [PartitionId]
    public Guid Id { get; init; }

    [Required]
    [Dimension(typeof(ReportingNode))]
    [IdentityProperty]
    [Display(Order = 10)]
    public string ReportingNode { get; init; }
}

For instance, the data pertaining to a certain [Reporting Node](#reporting-node) for a certain [Scenario](#scenario) is called an <code>IfrsPartition</code> and assigned a unique identifier.

In [0]:
public record PartitionByReportingNode : IfrsPartition {}

The partition </code>PartitionByReportingNodeAndPeriod</code> is a further partition of the <code>IfrsPartition</code> sets. In particular, it defines sets for the data pertaining to a certain [Reporting Node](#reporting-node), [Scenario](#scenario), year and month. The value of the <code>Month</code> is the last month of the reporting period to which the data belongs to. 

In [0]:
public record PartitionByReportingNodeAndPeriod : IfrsPartition {
    [Dimension(typeof(int), nameof(Year))]
    [IdentityProperty]
    [Display(Order = 20)]
    public int Year { get; init; }

    [Dimension(typeof(int), nameof(Month))]
    [IdentityProperty]
    [Display(Order = 30)]
    public int Month { get; init; }
    
    [Dimension(typeof(Scenario))]
    [IdentityProperty]
    [Display(Order = 40)]
    public string Scenario { get; init; }
}

<a id='policy-related-data-structures'></a>
# Policy-related Data Structures

The data structures below are the pieces of the data model where policy-related data is held.

<a id='data-node'></a>
## Data Node

A data node is basically a set of labels which identify data according to its:
- Reporting Node
- Scenario
- Contractual Currency
- FunctionalCurrency
- LineOfBusiness
- ValuationApproach
- OCI Type

In [0]:
public record DataNode : KeyedDimension, IPartitioned {
    [NotVisible]
    [PartitionKey(typeof(PartitionByReportingNode))]
    public Guid Partition { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(Currency))]
    //[Immutable]
    public string ContractualCurrency { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(Currency))]
    //[Immutable]
    public string FunctionalCurrency { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(LineOfBusiness))]
    //[Immutable]
    public string LineOfBusiness { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(ValuationApproach))]
    [Required]
    //[Immutable]
    public string ValuationApproach { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(OciType))]
    //[Immutable]
    public string OciType { get; init; }
}

<a id='portfolios'></a>
## Portfolios

A Portfolio is a set of Group of Contracts. The relationship is set on the Group of Contracts (see below).

Portfolios are of type DataNode and there are two types: Insurance Portfolio and Reinsurance Portfolios.

In [0]:
public record Portfolio : DataNode {}

public record InsurancePortfolio : Portfolio {}
public record ReinsurancePortfolio : Portfolio {}

<a id='group-of-contracts'></a>
## Group of Contracts

A Group of Contract is a set of contracts with the same Annual Cohort, Liability Type, Profitability and Partner (in case the contracts are done with a re-insurer).

Various Group of Contracts are put together into portfolios.

In [0]:
public record GroupOfContract : DataNode {
    [NotVisible]    
    [Dimension(typeof(int), nameof(AnnualCohort))]
    //[Immutable]
    public int AnnualCohort { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(LiabilityType))]
    //[Immutable]
    public string LiabilityType { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(Profitability))]
    //[Immutable]
    public string Profitability { get; init; }
 
    [Required]
    [NotVisible]    
    [Dimension(typeof(Portfolio))]
    //[Immutable]
    public string Portfolio { get; init; }

    [NotVisible]
    //[Immutable]
    public string YieldCurveName { get; init; }
    
    public virtual string Partner { get; init; }
}

A group of insurance contracts is called a **GIC** and has no information on Partner.

Only Groups of Reinsurance Contracts, defined below and usually called **GRIC**s, have Partner information.

In [0]:
public record GroupOfInsuranceContract : GroupOfContract {
    [Required]
    [NotVisible]    
    [Display(Name = "InsurancePortfolio")]
    [Dimension(typeof(InsurancePortfolio))]
    //[Immutable]
    public string Portfolio { get => base.Portfolio; init => base.Portfolio = value; }
    
    // TODO: for the case of internal reinsurance the Partner would be the reporting node, hence not null.
    // If this is true we need the [Required] attribute here, add some validation at dataNode import 
    // and to add logic in the GetNonPerformanceRiskRate method in ImportStorage.
    [NotVisible]    
    [NotMapped]
    //[Immutable]
    public override string Partner => null;
}

public record GroupOfReinsuranceContract : GroupOfContract {
    [Required]
    [NotVisible]    
    [Display(Name = "ReinsurancePortfolio")]
    [Dimension(typeof(ReinsurancePortfolio))]
    //[Immutable]
    public string Portfolio { get => base.Portfolio; init => base.Portfolio = value; }
}

<a id='data-node-state'></a>
## Data Node State

The Data State is used to track whether a Group of Contract is Active or Inactive across time (year and month). 
It is only possible to update the state of a Group of Contract from Active to Inactive.

In [0]:
public record DataNodeState : KeyedRecord, IPartitioned, IWithYearMonthAndScenario
{
    [NotVisible]
    [PartitionKey(typeof(PartitionByReportingNode))]
    public Guid Partition { get; init; }
    
    [Required]
    [IdentityProperty]
    [Dimension(typeof(GroupOfContract))]
    public string DataNode { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [DefaultValue(DefaultDataNodeActivationMonth)]
    public int Month { get; init; } = DefaultDataNodeActivationMonth;
    
    [Required]
    [DefaultValue(State.Active)]
    public State State { get; init; } = State.Active;

    [IdentityProperty]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

<a id='data-node-parameters'></a>
## Data Node Parameters

Data Node Parameters are used to keep track of other parameters pertaining to each Data Node, and their movements in time (year and month).

<br> <code>PremiumAllocation</code> : defines the weight of Premium to be included in the Experience Adjustement AoC Type of the Technical Margin and is valid only for Group of Insurance Contract with LiabilityType : Liability for Remaining Coverage.

<br> <code>CashFlowPeriodicity</code> : defines the periodicity of the provided cash flows, it is not a mandatory input column. Supported values can be found [here](../Constants/Enums#cashflowperiodicity). When the column *CashFlowPeriodicity* is missing from the input file it is assumed a monthly periodicity (default value). When the *CashFlowPeriodicity* column is present, a valid value must be entered. If the Cashflow file does not contain a specific CashflowPeriodicity, this parameter value is used for the whole of DataNode.

<br> <code>InterpolationMethod</code> : defines the interpolation method to be applied to interpolate the cash flow from non-default (monthly) periodicity. It is not a mandatory input column, unless CashflowPeriodicity column is provided. Supported values can be found [here](../Constants/Enums#interpolation-method). When the *InterpolationMethod* is not present in the input file and it is required because *CashFlowPeriodicity* is not monthly, then the default value for the *InterpolationMethod* is applied. The default value is 'NotApplicable'.

<br> <code>ReleasePattern</code> : defines the pattern of factors that will be used to defer and amortize any AmountType over the course of contract term. It is not a mandatory input.  This pattern is used, if no *ReleasePattern* is provided in the Cashflows (which is specific for combinations of AmountType and EstimateType). If provided, the ReleasePattern needs at least a column with 'ReleasePattern0' header. The 'ReleasePattern0' corresponds to the first value of the AnnualCohort which is defined in [GroupOfContracts](#group-of-contracts). Pattern for multiple years can be provided by adding columns 'ReleasePattern1', 'ReleasePattern2' etc. Even if the ReleasePattern header is present, the values for any given DataNode are still optional.

<br> <code>EconomicBasisDriver</code> : defines the economic basis that will be used in discounting any Release Pattern. It is not a mandatory input column. Supported values are the defined values for [EconomicBasis](#economic-basis). When the *EconomicBasisDriver* is not present in the input file, then the default value for the *EconomicBasisDriver* is applied. The default value is *Locked-in* rates for Building Block Approach, *Current* rates for Variable Fee Approach and Premium Allocation Approach for liabilities of incurred claims and *Nominal* for Premium Allocation Approach for liabilities of remaining coverage.

<br> <code>ReinsuranceCoverage</code> : defines the weight of the underlying gross business to be considered in the computation of the allocation of the Technical Margin in a Reinsurance case. In other words, it represents the percentage to which claims in the underlying GICs are expected to be reinboursed by the Reinsurance Contracts belonging to the GRIC. For proportional contracts, this factor is given by the cession while for other contracts it should be estimated. 

The latest Data Node Parameters available in the system with Year and Month earlier or equal to Year and Month of the closing period will be used as the current value during calculation.

In [0]:
public record DataNodeParameter : KeyedRecord, IPartitioned, IWithYearMonthAndScenario
{
    [NotVisible]
    [PartitionKey(typeof(PartitionByReportingNode))]
    public Guid Partition { get; init; }
        
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [DefaultValue(DefaultDataNodeActivationMonth)]
    public int Month { get; init; } = DefaultDataNodeActivationMonth;
        
    [Required]
    [IdentityProperty]
    [Dimension(typeof(GroupOfContract))]
    [Display(Order = 1)]
    public string DataNode { get; init; }

    [IdentityProperty]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

In [0]:
public record SingleDataNodeParameter : DataNodeParameter {
    [DefaultValue(DefaultPremiumExperienceAdjustmentFactor)]
    [Range(0, 1, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 20)]
    public double PremiumAllocation { get; init; } = DefaultPremiumExperienceAdjustmentFactor;

    [Dimension(typeof(CashFlowPeriodicity))]
    [Display(Order = 30)]
    public CashFlowPeriodicity CashFlowPeriodicity { get; init; }

    [Dimension(typeof(InterpolationMethod))]
    [Display(Order = 40)]
    public InterpolationMethod InterpolationMethod { get; init; }

    [Dimension(typeof(EconomicBasis))]
    [Display(Order = 50)]
    public string EconomicBasisDriver {get; init;}

    [Conversion(typeof(PrimitiveArrayConverter))]
    [Display(Order = 60)]
    public double[] ReleasePattern {get; init;}

}

public record InterDataNodeParameter : DataNodeParameter {
    [Required]
    [IdentityProperty]
    [Dimension(typeof(GroupOfContract))]
    [Display(Order = 10)]
    public string LinkedDataNode { get; init; }
    
    [Range(0, 1, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 20)]
    public double ReinsuranceCoverage { get; init; }
}

## DataNodeData
The <code>DataNodeData</code> type is not persisted (i.e. not stored in the database) and is used in order to hold Data Node data (i.e information from the dimension [data node](#data-node) and the data node [State](#data-node-state)) during data manipulations e.g import and report computations.

In [0]:
public record DataNodeData {
    public string DataNode { get; init; }
    
    //Portfolio
    public string ContractualCurrency { get; init; }
    public string FunctionalCurrency { get; init; }
    public string LineOfBusiness { get; init; }
    public string ValuationApproach { get; init; }
    public string OciType { get; init; }
    
    //GroupOfContract
    public string Portfolio { get; init; }
    public int AnnualCohort { get; init; }
    public string LiabilityType { get; init; }
    public string Profitability { get; init; }
    public string Partner { get; init; }
    public string YieldCurveName { get; init; }
    
    
    //DataNodeState
    public int Year { get; init; }
    public int Month { get; init; }
    public State State { get; init; }
    public State PreviousState { get; init; }
    
    public bool IsReinsurance { get; init; }

    public DataNodeData(){}

    public DataNodeData(GroupOfContract dn)
    {
        DataNode = dn.SystemName;
        ContractualCurrency  = dn.ContractualCurrency;
        FunctionalCurrency  = dn.FunctionalCurrency;
        LineOfBusiness  = dn.LineOfBusiness;
        ValuationApproach  = dn.ValuationApproach;
        OciType  = dn.OciType;
        Portfolio  = dn.Portfolio;
        AnnualCohort  = dn.AnnualCohort;
        LiabilityType  = dn.LiabilityType;
        Profitability  = dn.Profitability;
        Partner  = dn.Partner;
        IsReinsurance  = dn.GetType().Name == nameof(GroupOfReinsuranceContract);
        YieldCurveName = dn.YieldCurveName;
    }
}

<a id='raw-variables'></a>
## Raw Variables (Cash flows)

Raw Variables are used to store **cash flows** on the database during import, i.e. *after* mappings are applied and *before* calculations are applied.

They are stored on the database for both audit and re-calculation purposes.

In [0]:
public abstract record BaseVariableIdentity {
    [NotVisible]
    [Dimension(typeof(GroupOfContract))]
    [IdentityProperty]
    public string DataNode { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(AocType))]
    [IdentityProperty]
    public string AocType { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(Novelty))]
    [IdentityProperty]
    public string Novelty { get; init; }
}

In [0]:
public abstract record BaseDataRecord : BaseVariableIdentity, IKeyed, IPartitioned {
    [Key]
    [NotVisible]     
    public Guid Id { get; init; }
    
    [NotVisible]
    [PartitionKey(typeof(PartitionByReportingNodeAndPeriod))]
    public Guid Partition { get; init; }
    
    [Conversion(typeof(PrimitiveArrayConverter))]
    public double[] Values {get; set;}
    
    [NotVisible]    
    [Dimension(typeof(EstimateType))]
    [IdentityProperty]
    public string EstimateType { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(AmountType))]
    [IdentityProperty]
    public string AmountType { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(int),nameof(AccidentYear))]
    [IdentityProperty]
    public int? AccidentYear { get; init; }
}

Basically, <code>RawVariable</code> is defined for a certain Reporting Node, Scenario, Year, Month, Amount Type and Estimate Type.

In [0]:
public record RawVariable : BaseDataRecord {}

<a id='ifrs-variable'></a>
## Ifrs Variable

Ifrs Variables are used to store:

- **Actual Values**
- **Present Values**

Present Values are calculated from the [raw variables](#raw-variables) during the import of cash flows.

Ifrs Variables are stored on the database, i.e. they are 'persisted', for performance purposes.

In [0]:
public record IfrsVariable : BaseDataRecord
{
    [NotVisible]    
    [Dimension(typeof(EconomicBasis))]
    [IdentityProperty]
    public string EconomicBasis { get; init; }
    
    public IfrsVariable (){}
}

<a id='import-identity'></a>
## Import Identity
The Import Identity contains the data elements used to determine the data sets which are used when performing data manipulations, e.g. during a [data import](./Import/1ImportScope-Identities). In particular, an Import Identity is defined by: DataNode, AoC Type, Novelty, and whether it's reinsurance data or not.

In [0]:
public record ImportIdentity : BaseVariableIdentity {
      
    [NotVisible]
    public bool IsReinsurance { get; init; }
    
    [NotVisible]
    public string ValuationApproach { get; init; }

    [NotVisible]
    public string LiabilityType { get; init; }
    
    [NotVisible]
    public int ProjectionPeriod { get; init; }
    
    public AocStep AocStep => new AocStep(AocType, Novelty);
    
    public ImportScope ImportScope { get; init; }
    
    public ImportIdentity(RawVariable rv){
        DataNode = rv.DataNode;
        AocType = rv.AocType;
        Novelty = rv.Novelty;
    }
    
    public ImportIdentity(IfrsVariable iv){
        DataNode = iv.DataNode;
        AocType = iv.AocType;
        Novelty = iv.Novelty;
    }

    public ImportIdentity(){}
}

<a id='report-variable'></a>
# Report Variable

Report Variables are used to hold data after data manipulations and before being outputed in the form of a report. They are not persisted, i.e. they are not saved on the database.

In [0]:
public record ReportVariable {

    [NotVisible]
    [Dimension(typeof(ReportingNode))]
    [IdentityProperty]
    public string ReportingNode { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Scenario))]
    [IdentityProperty]
    public string Scenario { get; init; }

    [NotVisible]
    [Dimension(typeof(Currency))]
    [IdentityProperty]
    [AggregateBy]
    public string Currency { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Currency), nameof(FunctionalCurrency))]
    [IdentityProperty]
    public string FunctionalCurrency { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Currency), nameof(ContractualCurrency))]
    [IdentityProperty]
    public string ContractualCurrency { get; init; }
    
    [NotVisible]
    [Dimension(typeof(GroupOfContract))]
    [IdentityProperty]
    public string GroupOfContract { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Portfolio))]
    [IdentityProperty]
    public string Portfolio { get; init; }
    
    [NotVisible]
    [Dimension(typeof(LineOfBusiness))]
    [IdentityProperty]
    public string LineOfBusiness { get; init; }
    
    [NotVisible]
    [Dimension(typeof(LiabilityType))]
    [IdentityProperty]
    public string LiabilityType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Profitability), nameof(InitialProfitability))]
    [IdentityProperty]
    public string InitialProfitability { get; init; }
    
    [NotVisible]
    [Dimension(typeof(ValuationApproach))]
    [IdentityProperty]
    public string ValuationApproach { get; init; }
    
    [NotVisible]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(AnnualCohort))]
    [IdentityProperty]
    public int AnnualCohort { get; init; }
    
    [NotVisible]
    [Dimension(typeof(OciType))]
    [IdentityProperty]
    public string OciType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Partner))]
    [IdentityProperty]
    public string Partner { get; init; }
        
    [NotVisible]
    [IdentityProperty]
    public bool IsReinsurance { get; init; }
    
    [NotVisible]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(AccidentYear))]
    [IdentityProperty]
    public int AccidentYear { get; init; }

    [NotVisible]
    [Dimension(typeof(ServicePeriod))]
    [IdentityProperty]
    public ServicePeriod ServicePeriod { get; init; }

    [NotVisible]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(ProjectionConfiguration), nameof(Projection))]
    [IdentityProperty]
    public string Projection { get; init;}
    
    [NotVisible]
    [Dimension(typeof(VariableType))]
    [IdentityProperty]
    public string VariableType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Novelty))]
    [IdentityProperty]
    public string Novelty { get; init; }
    
    [NotVisible]
    [Dimension(typeof(AmountType))]
    [IdentityProperty]
    public string AmountType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(EstimateType))]
    [IdentityProperty]
    public string EstimateType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(EconomicBasis))]
    [IdentityProperty]
    public string EconomicBasis { get; init; }
    
    public double Value { get; init; }
    
    public ReportVariable(){}
    public ReportVariable(ReportVariable rv){
        ReportingNode = rv.ReportingNode;
        Scenario = rv.Scenario;
        Currency = rv.Currency;
        FunctionalCurrency = rv.FunctionalCurrency;
        ContractualCurrency = rv.ContractualCurrency;
        GroupOfContract = rv.GroupOfContract;
        Portfolio = rv.Portfolio;
        LineOfBusiness = rv.LineOfBusiness;
        LiabilityType = rv.LiabilityType;
        InitialProfitability = rv.InitialProfitability;
        ValuationApproach = rv.ValuationApproach;
        AnnualCohort = rv.AnnualCohort;
        OciType = rv.OciType;
        Partner = rv.Partner;
        IsReinsurance = rv.IsReinsurance;
        AccidentYear = rv.AccidentYear;
        ServicePeriod = rv.ServicePeriod;
        Projection = rv.Projection;
        VariableType = rv.VariableType;
        Novelty = rv.Novelty;
        AmountType = rv.AmountType;
        EstimateType = rv.EstimateType;
        EconomicBasis = rv.EconomicBasis;
        Value = rv.Value;
    }
}

# Args

Args are used to hold data related to the partition and additional information required to perform calculation at import and report. 
They are not persisted, i.e. they are not saved on the database.

In [0]:
public record Args
{
    [Required]
    [IdentityProperty]
    [Dimension(typeof(ReportingNode))]
    public string ReportingNode { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Year { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public int Month { get; init; } 
    
    [IdentityProperty]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }

    [IdentityProperty]
    public Periodicity Periodicity{ get; init; }

    public Args(string reportingNode, int year, int month, Periodicity periodicity, string scenario)
    {
        ReportingNode = reportingNode;
        Year = year;
        Month = month;
        Periodicity = periodicity;
        Scenario = scenario;
    }
}

In [0]:
public record ImportArgs : Args
{
    public string ImportFormat { get; init; }
       
    public ImportArgs(string reportingNode, int year, int month, Periodicity periodicity, string scenario, string importFormat)
        : base(reportingNode, year, month, periodicity, scenario)
    {
        ImportFormat = importFormat;
    }
}

In [0]:
public record ReportArgs : Args
{
    public string HierarchyName { get; init; }
    
    public CurrencyType CurrencyType { get; init; }
    
    public string ReportName { get; init; } // this is the key to which data to load (like loading behavior). If null, loads everything
    
    public ReportArgs(string reportingNode, int year, int month, Periodicity periodicity, string scenario, string hierarchyName, CurrencyType currencyType)
        : base(reportingNode, year, month, periodicity, scenario)
    {
        CurrencyType = currencyType;
        HierarchyName = hierarchyName;
    }
}